In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
import os
import os.path
from os import path
import librosa as lb
import librosa.display
import pandas as pd
from pathlib import Path
import numpy as np
import IPython.display as ipd
import csv
import scipy
import matplotlib.pyplot as plt
import datetime
import tensorflow as tf
import glob
import math
import warnings
import pickle
from sklearn.utils import shuffle
import zipfile

In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
tf.random.set_seed(999)
np.random.seed(999)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Set your drive's path for the files used.
maleVoicePath = '/content/drive/My Drive/ML Project/male'
whiteNoisePath = '/content/drive/My Drive/ML Project/whitenoise'
DrivePath = '/content/drive/My Drive/ML Project'
csvPath = '/content/drive/My Drive/ML Project/data.csv'
trainPath = '/content/drive/My Drive/ML Project/train'
numMaleFiles = 2715
limit = 1000

In [ ]:
# Static Variables for audio information
windowLength = 256
overlap      = round(0.25 * windowLength) # overlap of 75%
ffTLength    = windowLength
inputFs      = 48e3
fs           = 16e3
numFeatures  = ffTLength//2 + 1
numSegments  = 8

In [ ]:
class FeatureExtractor:
    def __init__(self, audio, *, windowLength, overlap, sample_rate):
        self.audio = audio
        self.ffT_length = windowLength
        self.window_length = windowLength
        self.overlap = overlap
        self.sample_rate = sample_rate
        self.window = scipy.signal.hamming(self.window_length, sym=False)

    def get_stft_spectrogram(self):
        return lb.stft(self.audio, n_fft=self.ffT_length, win_length=self.window_length, hop_length=self.overlap,
                            window=self.window, center=True)

    def get_audio_from_stft_spectrogram(self, stft_features):
        return lb.istft(stft_features, win_length=self.window_length, hop_length=self.overlap,
                             window=self.window, center=True)

    def get_mel_spectrogram(self):
        return lb.feature.melspectrogram(self.audio, sr=self.sample_rate, power=2.0, n_fft=self.ffT_length, hop_length=self.overlap)

    def get_audio_from_mel_spectrogram(self, M):
        return lb.feature.inverse.mel_to_audio(M, sr=self.sample_rate, n_fft=self.ffT_length, hop_length=self.overlap,
                                             win_length=self.window_length, window=self.window,
                                             center=True, pad_mode='reflect', power=2.0, n_iter=32, length=None)